In [ ]:
import clouddrift as cd

ds = cd.adapters.ibtracs.to_raggedarray("v04r01", "LAST_3_YEARS", "/home/ksantana/.clouddrift/tmp")
ds

In [ ]:
import numpy as np

# Select the data for the year 2022 only by using the ragged.subset method
year = 2023
ds = cd.ragged.subset(
    ds,
    {"time": (np.datetime64(f"{year}-01-01"), np.datetime64(f"{year}-12-31"))},
    row_dim_name="storm",
    rowsize_var_name="numobs"
)
ds

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

# make figure
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
hl = cd.plotting.plot_ragged(
    ax,
    ds.lon,
    ds.lat,
    ds.numobs,
    transform=ccrs.PlateCarree(),
)
ax.set_extent([-150, 0, 5, 70], crs=ccrs.PlateCarree())
ax.set_title(f"Hurricane Season {year}")
ax.coastlines()
ax.gridlines(draw_labels=True)